# GenAI Tracing and Evaluation with MLflow 3 

https://mlflow.org/docs/latest/genai/data-model/experiments 


In [2]:
%reload_ext autoreload
%autoreload 2

import getpass
import os
import sys
from pathlib import Path

import openai


### Secrets and Environment Variables

MLflow:<br>
`MLFLOW_TRACKING_URI`<br>

OpenAI API Key:<br>
`OPENAI_API_KEY`


In [5]:
os.environ["MLFLOW_TRACKING_URI"] = "http://0.0.0.0:5001"
os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter OPEN API key:")

### Check connection to MLflow


In [6]:
import mlflow 

# List experiments in MLflow
mlflow.search_experiments()

[<Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1750123536195, experiment_id='0', last_update_time=1750123536195, lifecycle_stage='active', name='Default', tags={}>]

In [7]:
# Set up MLflow experiment
mlflow.set_experiment("5-genai-with-mlflow-3")

2025/06/17 09:23:59 INFO mlflow.tracking.fluent: Experiment with name '5-genai-with-mlflow-3' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/238283046149206350', creation_time=1750127039368, experiment_id='238283046149206350', last_update_time=1750127039368, lifecycle_stage='active', name='5-genai-with-mlflow-3', tags={}>

# Example 1: Tracing a GenAI Application



https://mlflow.org/docs/latest/genai/getting-started/tracing/tracing-notebook 

## Automatic Tracing of LLM calls

In [8]:
# Enable MLflow's autologging to instrument your application with Tracing
mlflow.openai.autolog()

# Create an OpenAI client
client = openai.OpenAI()


# Use the trace decorator to capture the application's entry point
@mlflow.trace
def my_app(input: str):
    # This call is automatically instrumented by `mlflow.openai.autolog()`
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {
                "role": "system",
                "content": "You are a helpful assistant.",
            },
            {
                "role": "user",
                "content": input,
            },
        ],
    )
    return response.choices[0].message.content


my_app(input="What is MLflow?")

'MLflow is an open-source platform aimed at managing the machine learning lifecycle, including experimentation, reproducibility, and deployment. It was developed by Databricks and provides a set of tools to facilitate the following key components of the machine learning workflow:\n\n1. **Tracking**: MLflow allows users to log and query experiments to keep track of model parameters, metrics, and artifacts. You can log your training parameters, evaluation metrics, and the resulting models for later comparison and analysis.\n\n2. **Projects**: MLflow provides a way to organize and package machine learning code into reproducible projects. A project is defined by a directory that contains code and configuration files, making it easy to reproduce results on different machines.\n\n3. **Models**: MLflow has components for managing and deploying machine learning models. It supports multiple model formats and provides tools for versioning models, serving them via REST API, or deploying them to v

Trace(trace_id=c849288e49e3463d8a9d4f69c3520707)

## Tracing LangChain🦜⛓️

https://mlflow.org/docs/latest/genai/tracing/integrations/listing/langchain 


In [9]:
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI


# Enabling autolog for LangChain will enable trace logging.
mlflow.langchain.autolog()

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.7, max_tokens=1000)

prompt_template = PromptTemplate.from_template(
    "Answer the question as if you are {person}, fully embodying their style, wit, personality, and habits of speech. "
    "Emulate their quirks and mannerisms to the best of your ability, embracing their traits—even if they aren't entirely "
    "constructive or inoffensive. The question is: {question}"
)

chain = prompt_template | llm | StrOutputParser()

# Let's test another call
chain.invoke(
    {
        "person": "Linus Torvalds",
        "question": "Can I just set everyone's access to sudo to make things easier?",
    }
)

'Oh, come on! Really? You want to give everyone sudo access? That\'s like handing the keys to the nuclear launch codes to your cat because it seems convenient. Sure, it might make things "easier," but at what cost? \n\nLet’s be honest here: giving everyone sudo access is like saying, "Hey, everyone, why don\'t you just go ahead and mess with system files at will?" It’s the kind of decision that’s going to come back and bite you in the posterior. You think it’s easier now, but when someone accidentally wipes out the entire system because they thought `rm -rf /` was a good idea, you’ll be the one left picking up the pieces.\n\nYes, managing permissions can be a hassle. You might have to do a little bit of thinking and a little bit of configuration. But that’s part of being a responsible sysadmin! You want to maintain a balance between usability and security. There are ways to delegate privileges without giving everyone the power to shoot themselves in the foot. \n\nSo, do yourself a favo

Trace(trace_id=626793512f8847ceb973ea3a6fab3590)

## Token Usage Tracking

https://mlflow.org/docs/latest/genai/tracing/integrations/listing/langchain#token-usage-tracking


In [10]:
# Execute the chain defined in the previous example
chain.invoke(
    {
        "person": "Linus Torvalds",
        "question": "Can I just set everyone's access to sudo to make things easier?",
    }
)

# Get the trace object just created
last_trace_id = mlflow.get_last_active_trace_id()
trace = mlflow.get_trace(trace_id=last_trace_id)

# Print the token usage
total_usage = trace.info.token_usage
print("== Total token usage: ==")
print(f"  Input tokens: {total_usage['input_tokens']}")
print(f"  Output tokens: {total_usage['output_tokens']}")
print(f"  Total tokens: {total_usage['total_tokens']}")

# Print the token usage for each LLM call
print("\n== Token usage for each LLM call: ==")
for span in trace.data.spans:
    if usage := span.get_attribute("mlflow.chat.tokenUsage"):
        print(f"{span.name}:")
        print(f"  Input tokens: {usage['input_tokens']}")
        print(f"  Output tokens: {usage['output_tokens']}")
        print(f"  Total tokens: {usage['total_tokens']}")

== Total token usage: ==
  Input tokens: 81
  Output tokens: 277
  Total tokens: 358

== Token usage for each LLM call: ==
ChatOpenAI:
  Input tokens: 81
  Output tokens: 277
  Total tokens: 358
Completions:
  Input tokens: 81
  Output tokens: 277
  Total tokens: 358


Trace(trace_id=b420c9e925f349bbb59a3f2cf86e5adb)

# Example 2:  Tracing LangGraph🦜🕸️

https://mlflow.org/docs/latest/genai/tracing/integrations/listing/langgraph 


In [12]:
from typing import Literal

import mlflow

from langchain_core.messages import AIMessage, ToolCall
from langchain_core.outputs import ChatGeneration, ChatResult
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI
from langgraph.prebuilt import create_react_agent

# Enabling tracing for LangGraph (LangChain)
mlflow.langchain.autolog()


@tool
def get_weather(city: Literal["nyc", "sf"]):
    """Use this to get weather information."""
    if city == "nyc":
        return "It might be cloudy in nyc"
    elif city == "sf":
        return "It's always sunny in sf"


llm = ChatOpenAI(model="gpt-4o-mini")
tools = [get_weather]
graph = create_react_agent(llm, tools)

# Invoke the graph
result = graph.invoke(
    {"messages": [{"role": "user", "content": "what is the weather in sf?"}]}
)

Trace(trace_id=c35f6092d0d243bdaca3c2fe8adee83b)

# Example 3: Prompt Management

https://mlflow.org/docs/latest/genai/mlflow-3/genai-agent

In [14]:

system_prompt = mlflow.genai.register_prompt(
    name="chatbot_prompt",
    template="You are a chatbot that can answer questions about IT. Answer this question: {{question}}",
    commit_message="Initial version of chatbot",
)

2025/06/17 09:24:36 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for prompt version to finish creation. Prompt name: chatbot_prompt, version 3


In [15]:
from langchain.schema.output_parser import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_template(system_prompt.to_single_brace_format())
chain = prompt | ChatOpenAI(temperature=0.7) | StrOutputParser()
question = "What is MLflow?"
print(chain.invoke({"question": question}))
# MLflow is an open-source platform for managing the end-to-end machine learning lifecycle...

MLflow is an open-source platform for managing the end-to-end machine learning lifecycle. It includes tools for tracking experiments, packaging code into reproducible runs, and sharing and deploying models. MLflow is designed to work with any machine learning library and language, and it provides integration with popular tools and frameworks such as TensorFlow, PyTorch, and scikit-learn.


Trace(trace_id=bdb9ab32785c4984adf5b788986c2047)

In [16]:
# set the active model for linking traces
mlflow.set_active_model(name="langchain_model")

# Enable autologging so that interactive traces from the client are automatically linked to a LoggedModel
mlflow.langchain.autolog()

questions = [
    "What is MLflow Tracking and how does it work?",
    "What is Unity Catalog?",
    "What are user-defined functions (UDFs)?",
]
outputs = []

for question in questions:
    outputs.append(chain.invoke({"question": question}))

# fetch the current active model's id and check traces
active_model_id = mlflow.get_active_model_id()
mlflow.search_traces(model_id=active_model_id)
#                            trace_id                                             trace  ...  assessments                        request_id
# 0  e807ab0a020f4794989a24c84c2892ad  Trace(trace_id=e807ab0a020f4794989a24c84c2892ad)  ...           []  e807ab0a020f4794989a24c84c2892ad
# 1  4eb83e4adb6a4f3494bc5b33aca4e970  Trace(trace_id=4eb83e4adb6a4f3494bc5b33aca4e970)  ...           []  4eb83e4adb6a4f3494bc5b33aca4e970
# 2  42b100851f934c969c352930f699308d  Trace(trace_id=42b100851f934c969c352930f699308d)  ...           []  42b100851f934c969c352930f699308d

2025/06/17 09:24:40 INFO mlflow.tracking.fluent: LoggedModel with name 'langchain_model' does not exist, creating one...
2025/06/17 09:24:40 INFO mlflow.tracking.fluent: Active model is set to the logged model with ID: m-8aecef794c88486eab33ce4882d2f325


,trace_id,trace,client_request_id,state,request_time,execution_duration,request,response,trace_metadata,tags,spans,assessments
0,a3bb0d517b78419da40b7760196bd421,Trace(trace_id=a3bb0d517b78419da40b7760196bd421),None,TraceState.OK,1750127083550,1169,{'question': 'What are user-defined functions ...,User-defined functions (UDFs) are functions th...,"{'mlflow.trace.tokenUsage': '{""input_tokens"": ...",{'mlflow.artifactLocation': 'mlflow-artifacts:...,"[{'trace_id': 'eZPcJvtkZKSBMWFQYqcWcw==', 'spa...",[]
1,b93ace222b444400990ad4c1536cbb11,Trace(trace_id=b93ace222b444400990ad4c1536cbb11),None,TraceState.OK,1750127082463,1066,{'question': 'What is Unity Catalog?'},Unity Catalog is a feature in the Unity game d...,"{'mlflow.trace.tokenUsage': '{""input_tokens"": ...",{'mlflow.artifactLocation': 'mlflow-artifacts:...,"[{'trace_id': 'MCEt3sgGabLVl76ntF8dHw==', 'spa...",[]
2,c3b1262a4f924cb38beca16ec39eac24,Trace(trace_id=c3b1262a4f924cb38beca16ec39eac24),None,TraceState.OK,1750127080728,1714,{'question': 'What is MLflow Tracking and how ...,MLflow Tracking is a component of the MLflow o...,"{'mlflow.trace.tokenUsage': '{""input_tokens"": ...",{'mlflow.artifactLocation': 'mlflow-artifacts:...,"[{'trace_id': '6EBUJEXhv7F7hTdyKZeWKg==', 'spa...",[]


[Trace(trace_id=c3b1262a4f924cb38beca16ec39eac24), Trace(trace_id=b93ace222b444400990ad4c1536cbb11), Trace(trace_id=a3bb0d517b78419da40b7760196bd421)]

# Example 4:Evaluate the agent's performance

In [17]:
# Prepare the eval dataset in a pandas DataFrame
import pandas as pd

eval_df = pd.DataFrame(
    {
        "messages": questions,
        "expected_response": [
            """MLflow Tracking is a key component of the MLflow platform designed to record and manage machine learning experiments. It enables data scientists and engineers to log parameters, code versions, metrics, and artifacts in a systematic way, facilitating experiment tracking and reproducibility.\n\nHow It Works:\n\nAt the heart of MLflow Tracking is the concept of a run, which is an execution of a machine learning code. Each run can log the following:\n\nParameters: Input variables or hyperparameters used in the model (e.g., learning rate, number of trees). Metrics: Quantitative measures to evaluate the model's performance (e.g., accuracy, loss). Artifacts: Output files like models, datasets, or images generated during the run. Source Code: The version of the code or Git commit hash used. These logs are stored in a tracking server, which can be set up locally or on a remote server. The tracking server uses a backend storage (like a database or file system) to keep a record of all runs and their associated data.\n\n Users interact with MLflow Tracking through its APIs available in multiple languages (Python, R, Java, etc.). By invoking these APIs in the code, you can start and end runs, and log data as the experiment progresses. Additionally, MLflow offers autologging capabilities for popular machine learning libraries, automatically capturing relevant parameters and metrics without manual code changes.\n\nThe logged data can be visualized using the MLflow UI, a web-based interface that displays all experiments and runs. This UI allows you to compare runs side-by-side, filter results, and analyze performance metrics over time. It aids in identifying the best models and understanding the impact of different parameters.\n\nBy providing a structured way to record experiments, MLflow Tracking enhances collaboration among team members, ensures transparency, and makes it easier to reproduce results. It integrates seamlessly with other MLflow components like Projects and Model Registry, offering a comprehensive solution for managing the machine learning lifecycle.""",
            """Unity Catalog is a feature in Databricks that allows you to create a centralized inventory of your data assets, such as tables, views, and functions, and share them across different teams and projects. It enables easy discovery, collaboration, and reuse of data assets within your organization.\n\nWith Unity Catalog, you can:\n\n1. Create a single source of truth for your data assets: Unity Catalog acts as a central repository of all your data assets, making it easier to find and access the data you need.\n2. Improve collaboration: By providing a shared inventory of data assets, Unity Catalog enables data scientists, engineers, and other stakeholders to collaborate more effectively.\n3. Foster reuse of data assets: Unity Catalog encourages the reuse of existing data assets, reducing the need to create new assets from scratch and improving overall efficiency.\n4. Enhance data governance: Unity Catalog provides a clear view of data assets, enabling better data governance and compliance.\n\nUnity Catalog is particularly useful in large organizations where data is scattered across different teams, projects, and environments. It helps create a unified view of data assets, making it easier to work with data across different teams and projects.""",
            """User-defined functions (UDFs) in the context of Databricks and Apache Spark are custom functions that you can create to perform specific tasks on your data. These functions are written in a programming language such as Python, Java, Scala, or SQL, and can be used to extend the built-in functionality of Spark.\n\nUDFs can be used to perform complex data transformations, data cleaning, or to apply custom business logic to your data. Once defined, UDFs can be invoked in SQL queries or in DataFrame transformations, allowing you to reuse your custom logic across multiple queries and applications.\n\nTo use UDFs in Databricks, you first need to define them in a supported programming language, and then register them with the SparkSession. Once registered, UDFs can be used in SQL queries or DataFrame transformations like any other built-in function.\n\nHere\'s an example of how to define and register a UDF in Python:\n\n```python\nfrom pyspark.sql.functions import udf\nfrom pyspark.sql.types import IntegerType\n\n# Define the UDF function\ndef multiply_by_two(value):\n    return value * 2\n\n# Register the UDF with the SparkSession\nmultiply_udf = udf(multiply_by_two, IntegerType())\n\n# Use the UDF in a DataFrame transformation\ndata = spark.range(10)\nresult = data.withColumn("multiplied", multiply_udf(data.id))\nresult.show()\n```\n\nIn this example, we define a UDF called `multiply_by_two` that multiplies a given value by two. We then register this UDF with the SparkSession using the `udf` function, and use it in a DataFrame transformation to multiply the `id` column of a DataFrame by two.""",
        ],
        "predictions": outputs,
    }
)

# Start a run to represent the evaluation job
with mlflow.start_run() as evaluation_run:
    eval_dataset = mlflow.data.from_pandas(
        df=eval_df,
        name="eval_dataset",
        targets="expected_response",
        predictions="predictions",
    )
    mlflow.log_input(dataset=eval_dataset)
    # Run the evaluation based on extra metrics
    # Current active model will be automatically used
    result = mlflow.evaluate(
        data=eval_dataset,
        extra_metrics=[
            mlflow.metrics.genai.answer_correctness("openai:/gpt-4o"),
            mlflow.metrics.genai.answer_relevance("openai:/gpt-4o"),
        ],
        # This is needed since answer_correctness looks for 'inputs' field
        evaluator_config={"col_mapping": {"inputs": "messages"}},
    )

result.tables["eval_results_table"]
#                                         messages  ...                  answer_relevance/v1/justification
# 0  What is MLflow Tracking and how does it work?  ...  The output directly addresses the input questi...
# 1                         What is Unity Catalog?  ...  The output is completely irrelevant to the inp...
# 2        What are user-defined functions (UDFs)?  ...  The output directly addresses the input questi...

2025/06/17 09:24:44 INFO mlflow.system_metrics.system_metrics_monitor: Skip logging GPU metrics. Set logger level to DEBUG for more details.
2025/06/17 09:24:44 INFO mlflow.system_metrics.system_metrics_monitor: Started monitoring system metrics.
2025/06/17 09:24:45 INFO mlflow.tracking.fluent: Active model is set to the logged model with ID: m-8aecef794c88486eab33ce4882d2f325
2025/06/17 09:24:45 INFO mlflow.tracking.fluent: Use `mlflow.set_active_model` to set the active model to a different one if needed.
2025/06/17 09:24:45 INFO mlflow.models.evaluation.default_evaluator: Testing metrics on first row...
/Users/mikhailrozhkov/dev/mlrepa/mlflow-1-metrics-tracking/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
100%|██████████| 3/3 [00:02<00:00,  1.12it/s]
2025/06/17 09:24:57 INFO mlflow.system_me

🏃 View run orderly-frog-315 at: http://0.0.0.0:5001/#/experiments/238283046149206350/runs/b500f434378d4e23a1affbd23dd4a28e
🧪 View experiment at: http://0.0.0.0:5001/#/experiments/238283046149206350


,messages,expected_response,predictions,answer_correctness/v1/score,answer_correctness/v1/justification,answer_relevance/v1/score,answer_relevance/v1/justification
0,What is MLflow Tracking and how does it work?,MLflow Tracking is a key component of the MLfl...,MLflow Tracking is a component of the MLflow o...,5,The output is correct and demonstrates a high ...,5,The output directly addresses the input questi...
1,What is Unity Catalog?,Unity Catalog is a feature in Databricks that ...,Unity Catalog is a feature in the Unity game d...,1,The output is completely incorrect. It describ...,1,The output is completely irrelevant to the inp...
2,What are user-defined functions (UDFs)?,User-defined functions (UDFs) in the context o...,User-defined functions (UDFs) are functions th...,3,The output addresses some aspects of user-defi...,5,The output directly addresses the input questi...
